# Colab 2/3 — Darwin Evolution (Champion-Suche)

**Rolle:** KI-Labor — genetische Evolution des Champion-Bots

**Dieses Notebook tut NUR:**
- Rohdaten von Drive laden (von Notebook 1 vorbereitet)
- `darwin_engine.run_multiverse()` ausfuehren
- Champion-Genotyp auf Drive speichern
- Telegram-Benachrichtigung senden

**Kein GPU noetig! Wähle: Runtime > None (CPU)**

Darwin nutzt Numba-JIT + joblib Parallel auf CPU-Kernen.
GPU wuerde hier nichts bringen - Evolution ist kein Deep Learning.

---
**Voraussetzung:** Notebook 1 muss vorher gelaufen sein
(Daten auf Drive: `MyDrive/BITCOIN4Traders/data/BTC_USDT_1h_raw.parquet`)
---

## Schritt 1: Setup

In [ ]:
import os, shutil, gc

PROJECT_DIR = '/content/BITCOIN4Traders'
REPO_URL    = 'https://github.com/juancarlosrial76-code/BITCOIN4Traders.git'

if os.path.exists(PROJECT_DIR) and not os.path.exists(f'{PROJECT_DIR}/.git'):
    shutil.rmtree(PROJECT_DIR)

if not os.path.exists(PROJECT_DIR):
    !git clone {REPO_URL} {PROJECT_DIR} --quiet
    print('Repo geklont.')
else:
    !git -C {PROJECT_DIR} pull --quiet
    print('Repo aktualisiert.')

os.chdir(PROJECT_DIR)

In [ ]:
%%time
# Evolution braucht kein torch - deutlich schnellere Installation
!pip install -q ccxt loguru pyarrow pandas numpy numba joblib tqdm pyyaml python-dotenv requests
print('Dependencies bereit.')

## Schritt 2: Drive mounten & Konfiguration

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

DRIVE_DIR  = '/content/drive/MyDrive/BITCOIN4Traders'
DRIVE_DATA = f'{DRIVE_DIR}/data'
DRIVE_PROC = f'{DRIVE_DIR}/processed'

import os
os.makedirs(DRIVE_DATA, exist_ok=True)

# ===== EVOLUTION-EINSTELLUNGEN =====
MV_GENERATIONS  = 15    # Generationen (mehr = robuster, aber laenger)
MV_POP_SIZE     = 20    # Bots pro Generation
MV_MC_SCENARIOS = 50    # Monte Carlo Szenarien (3 Regime x 50 = 150 Zeitlinien)
MV_MAX_DD       = 0.20  # Max Drawdown in JEDEM Szenario
MV_N_BARS       = 3000  # Mehr Daten = robustere Selektion
MV_AUTO_LOAD    = True  # Besten gespeicherten Champion laden & verbessern

print(f'Generationen: {MV_GENERATIONS}')
print(f'Population:   {MV_POP_SIZE}')
print(f'MC-Szenarien: {MV_MC_SCENARIOS * 3} (3 Regime x {MV_MC_SCENARIOS})')
print(f'Max Drawdown: {MV_MAX_DD * 100:.0f}%')

## Schritt 3: Rohdaten laden (von Notebook 1)

In [ ]:
import pandas as pd
from pathlib import Path
from loguru import logger

raw_file = Path(DRIVE_DATA) / 'BTC_USDT_1h_raw.parquet'

if raw_file.exists():
    price_data = pd.read_parquet(raw_file)
    # Letzten MV_N_BARS Bars behalten (Evolution braucht keine Jahre)
    if len(price_data) > MV_N_BARS:
        price_data = price_data.iloc[-MV_N_BARS:]
    logger.success(f'Daten von Drive: {len(price_data):,} Bars')
    print(f'Zeitraum: {price_data.index[0]} bis {price_data.index[-1]}')
else:
    logger.warning('Keine Daten auf Drive. Lade synthetische BTC-Daten (Fallback)...')
    import sys
    sys.path.insert(0, PROJECT_DIR)
    from darwin_engine import generate_synthetic_btc
    price_data = generate_synthetic_btc(n_bars=MV_N_BARS, seed=42)
    logger.info(f'Synthetische Daten: {len(price_data):,} Bars')
    logger.info('Bitte zuerst Notebook 1 ausfuehren fuer echte Marktdaten!')

## Schritt 4: Darwin Multiverse Evolution starten

In [ ]:
import sys, importlib.util
import gc

# Darwin Engine laden
sys.path.insert(0, PROJECT_DIR)
_spec = importlib.util.spec_from_file_location('darwin_engine', f'{PROJECT_DIR}/darwin_engine.py')
_mod  = importlib.util.module_from_spec(_spec)
_spec.loader.exec_module(_mod)
sys.modules['darwin_engine'] = _mod

from darwin_engine import run_multiverse, ChampionPersistence, TelegramNotifier

# Telegram (optional - sendet Benachrichtigung wenn fertig)
notifier = TelegramNotifier.from_env()

print('Starte Darwin Multiverse Evolution...')
print(f'Generationen: {MV_GENERATIONS} | Population: {MV_POP_SIZE} | MC: {MV_MC_SCENARIOS}')
print(f'Szenarien: flash_crash, slow_bear, sideways_hell, parabolic_run + {MV_MC_SCENARIOS}x bull/bear/chop')
print('=' * 70)

champion = run_multiverse(
    symbol='BTC/USDT',
    timeframe='1h',
    n_bars=len(price_data),
    generations=MV_GENERATIONS,
    pop_size=MV_POP_SIZE,
    n_mc_scenarios=MV_MC_SCENARIOS,
    max_dd_threshold=MV_MAX_DD,
    auto_load_champion=MV_AUTO_LOAD,
    save_dir=DRIVE_DATA,           # Direkt auf Drive speichern
    exchange_id='binance',
    seed=42,
)

gc.collect()

if champion is not None:
    print(f'\nChampion: {champion.name}')
    print(f'Typ:      {type(champion).__name__}')
    meta_path = f'{DRIVE_DATA}/multiverse_champion_meta.json'
    ChampionPersistence.print_meta(meta_path)
    try:
        notifier.send(f'Darwin Evolution abgeschlossen!\nChampion: {champion.name}')
    except Exception:
        pass
else:
    print('Kein Champion hat alle Szenarien ueberlebt.')
    print('Tipp: MV_MAX_DD auf 0.25 erhoehen oder MV_GENERATIONS auf 20.')

## Schritt 5: Risk Engine pruefen (optional)

In [ ]:
# Nur ausfuehren wenn champion gefunden
if champion is not None:
    import importlib.util
    _rspec = importlib.util.spec_from_file_location('risk_engine', f'{PROJECT_DIR}/risk_engine.py')
    _rmod  = importlib.util.module_from_spec(_rspec)
    _rspec.loader.exec_module(_rmod)
    sys.modules['risk_engine'] = _rmod

    from risk_engine import run_full_pipeline, RiskConfig

    risk_cfg = RiskConfig(
        initial_capital=10_000.0,
        risk_per_trade=0.01,
        kelly_fraction=0.5,
        rr_ratio=2.0,
        atr_sl_multiplier=2.0,
        fee_rate=0.001,
        slippage_rate=0.0005,
        max_daily_loss_pct=0.05,
        max_consecutive_losses=3,
        max_drawdown_pct=0.15,
        min_capital_pct=0.30,
    )

    report = run_full_pipeline(
        symbol='BTC/USDT',
        timeframe='1h',
        n_bars=1500,
        risk_config=risk_cfg,
        n_generations=5,
        population_size=16,
        n_wfv_splits=3,
    )
    if report:
        report.print_summary()
    else:
        print('Kein Champion hat die 6 Sicherheits-Gates bestanden.')
else:
    print('Kein Champion -> Risk Engine wird uebersprungen.')

## Fertig!

Der Champion-Genotyp liegt jetzt auf:
- `MyDrive/BITCOIN4Traders/data/multiverse_champion.pkl`
- `MyDrive/BITCOIN4Traders/data/multiverse_champion_meta.json`

Naechste Schritte:
- **Colab 3** starten fuer PPO Deep-Learning Training (braucht GPU T4)
- Oder: Der Linux Local Master (PM2) wird den Champion beim naechsten sync_champion.sh-Run automatisch von GitHub holen